![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [23]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf, grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.graphics.api as smg

qb = QuantBook()
start_time = datetime(2020, 1, 5) # start datetime for history call
end_time = datetime(2020, 12, 3) # end datetime for history call

In [3]:
    #Dickey-Fuller test stolen from https://www.hackdeploy.com/augmented-dickey-fuller-test-in-python/
    def ADF_Stationarity_Test(timeseries, significance_level=0.05, printResults = True):
        is_stationary = None
        
        adf_test = adfuller(timeseries, autolag='AIC')
        pValue = adf_test[1]
        
        if (pValue<significance_level):
            is_stationary = True
        else:
            is_stationary = False
        
        if printResults:
            dfResults = pd.Series(adf_test[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adf_test[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)
            print('Is it stationary? '+str(is_stationary))
        else:
            return is_stationary

In [4]:
forex = qb.AddForex("USDCAD", Resolution.Daily, Market.Oanda).Symbol
forex_data = qb.History(forex, start_time, end_time)
print(forex_data['close'].head())
print(forex_data['close'].tail())
print(forex_data['close'].size)
forex_data['close'].plot(title='USDCAD')

In [5]:
oil = qb.AddCfd("WTICOUSD", Resolution.Daily, Market.Oanda).Symbol
oil_data = qb.History(oil, start_time, end_time)
print(oil_data['close'].head())
print(oil_data['close'].tail())
print(oil_data['close'].size)
plt.gca().invert_yaxis()
oil_data['close'].plot(title='Crude Oil Inverted')


In [25]:
plt.scatter(oil_data['close'].values, forex_data['close'].values)
plt.show()

corr = np.corrcoef(oil_data['close'].values, forex_data['close'])
print("Correlation between time series is ", corr[0][1])

In [7]:
print('Running Augmented Dickey-Fuller Test for Crude Oil...\n')
ADF_Stationarity_Test(oil_data['close'].values)

print('\n\n\n')

print('Running Augmented Dickey-Fuller Test for Forex...\n')
ADF_Stationarity_Test(forex_data['close'].values)

In [8]:
# Note: Will import training data later.
"""
# We need to split the series into training and testing data
n_obs=80
X_train_oil, X_test_oil = oil_data['close'][0:-n_obs], oil_data['close'][-n_obs:]
print("Oil:")
print(X_train_oil.shape, X_test_oil.shape)

print()

# We need to split the series into training and testing data
n_obs=80
X_train_forex, X_test_forex = forex_data['close'][0:-n_obs], oil_data['close'][-n_obs:]
print("Cad")
print(X_train_forex.shape, X_test_forex.shape)

# Difference the training data to make it stationary

train_oil = X_train_oil.dropna().values.tolist()
train_forex = X_train_forex.dropna().values.tolist()

stationary_oil_data = pd.DataFrame(difference(train_oil))
stationary_forex_data = pd.DataFrame(difference(train_forex))

stationary_oil_data.plot(title='Stationary Crude Oil')
stationary_forex_data.plot(title='Stationary Forex')
"""
print('Wow')

In [9]:
# create a differenced series
def difference(series, interval=1):
    diff = []
    for i in range(interval, len(series)):
        value = series[i] - series[i - interval]
        diff.append(value)
    return diff

In [10]:
print('Making data stationary...\n')
stationary_oil_data = pd.DataFrame(difference(oil_data['close']))
stationary_forex_data = pd.DataFrame(difference(forex_data['close']))

stationary_oil_data.plot(title='Stationary Crude Oil')
stationary_forex_data.plot(title='Stationary Forex')

In [11]:
print('Running Augmented Dickey-Fuller Test for Stationary Crude Oil...\n')
ADF_Stationarity_Test(stationary_oil_data.values)

print('\n\n\n')

print('Running Augmented Dickey-Fuller Test for Stationary Crude Oil...\n')
ADF_Stationarity_Test(stationary_forex_data.values)

In [12]:
# Testing for cointegration
print('Testing for Cointegration...\n')
statsmodels.tsa.stattools.coint(stationary_oil_data, stationary_forex_data, trend='c', method='aeg', maxlag=None, autolag='aic', return_results=None)

In [13]:
oil_cad = pd.DataFrame({'Oil': np.squeeze(stationary_oil_data),
                        'Cad': np.squeeze(stationary_forex_data)})

print('Stationary Oil vs Cad')
oil_cad.head()

In [14]:
# https://medium.com/swlh/using-granger-causality-test-to-know-if-one-time-series-is-impacting-in-predicting-another-6285b9fd2d1c
print('Running Granger Causality Test...\n')
granger_test = grangercausalitytests(oil_cad, maxlag=15)

In [21]:
# Auto correlation test on raw data
auto_corr_oil = plot_acf(oil_data['close'].values)
auto_corr_oil.show()

auto_corr_cad = plot_acf(forex_data['close'].values)
auto_corr_cad.show()

In [ ]:
# Cross correlation test on raw data
cross_corr_oil = plot_pacf(forex_data['close'].values)
cross_corr_oil.show()

cross_corr_cad = plot_pacf(forex_data['close'].values)
cross_corr_cad.show()

In [35]:
wow = plt.xcorr(oil_data['close'].values, forex_data['close'].values,  usevlines=True, maxlags=25)
plt.show()

In [17]:
"""
ARIMAX:
https://pyflux.readthedocs.io/en/latest/arimax.html
http://barnesanalytics.com/analyzing-multivariate-time-series-using-arimax-in-python-with-statsmodels
https://www.kaggle.com/viridisquotient/arimax

Arima:
https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/
"""

"""
Maybe start with 1 ar term.
1,0,0
"""

print('Wow')